In [1]:
import random
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Sequential
import roman_numerals as cnv
import numpy as np
import random

In [2]:
from encoder import CharacterTable 

# Parameters for the model and dataset.
DATASET_SIZE = 500
INPUT = 14
OUTPUT = 4
# object to encode roman numbers to one-hot 
romans = 'MDCLXVI '
rtable = CharacterTable(romans)

# object to encode arabic numbers to one-hot
chars = '0123456789 '
dtable = CharacterTable(chars)

In [3]:
seq_samples = []
seq_labels = []
used = []
repetitions = 0
generated = 0
print('Generating data...')
while len(seq_samples) < DATASET_SIZE:
    number = random.randint(1,2000)
    # skip if already in the dataset
    if number in used: continue
    used.append(number)
    # roman input
    roman = cnv.convert(number) 
    roman += ' ' * (INPUT - len(roman))
    
    # arabic output
    arabic = str(number)
    arabic += ' ' * (OUTPUT - len(arabic)) 
    
    seq_samples.append(roman)
    seq_labels.append(arabic)
print('Total roman numbers:', len(seq_samples))
#print("samples",seq_samples)
#print("labels",seq_labels)


Generating data...
Total roman numbers: 500


In [4]:
# one-hot encoding of all romans and arabic numbers
print('Vectorization...')
samples = np.zeros((DATASET_SIZE, INPUT, len(romans)), dtype=np.bool)
labels = np.zeros((DATASET_SIZE, OUTPUT, len(chars)), dtype=np.bool)
for i, sentence in enumerate(seq_samples):
    samples[i] = rtable.encode(sentence, INPUT)
for i, sentence in enumerate(seq_labels):
    labels[i] = dtable.encode(sentence, OUTPUT)
print("samples",samples.shape)    
print("labels",labels.shape)

Vectorization...
samples (500, 14, 8)
labels (500, 4, 11)


In [5]:
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples,labels,random_state=42)
print("Training samples",len(trainSamples))
print("Test samples",len(testSamples))

Training samples 375
Test samples 125


In [6]:
# Build the network
print('Build model...')
model = Sequential()
model.add(layers.LSTM(128, input_shape=(INPUT, len(romans))))
model.add(layers.RepeatVector(OUTPUT))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               70144     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 11)             1419      
Total params: 203,147
Trainable params: 203,147
Non-trainable params: 0
_________________________________________________________________


In [7]:
def avg(list):
    return sum(list)/len(list)

EPOCHS = 50
# Train the model each generation and show predictions against the validation dataset.
for iteration in range(1, 500):
    print()
    print('-' * 50)
    print('Iteration', iteration*EPOCHS)
    H = model.fit(trainSamples, trainLabels, batch_size=128, epochs=EPOCHS, 
                  validation_data=(testSamples, testLabels), verbose=0)
    print("Validation accuracy: {}".format(avg(H.history["val_accuracy"])))
    # Select 10 samples from the validation set at random to visualize errors.
    accuracy = 0
    for i in range(len(testSamples)):
        
        rowx, rowy = testSamples[np.array([i])], testLabels[np.array([i])]
        preds = model.predict_classes(rowx, verbose=0)
        roman = rtable.decode(rowx[0])
        correct = dtable.decode(rowy[0])
        prediction = dtable.decode(preds[0], calc_argmax=False)
        correctTxt = ""
        if correct == prediction:
            correctTxt+='OK!'
            accuracy+=1
        v = random.random()
        if v>0.9:
            print('expression:', roman, end=' ')
            print('correct:', correct, end=' ')
            print('predicted:', prediction, end=' ')
            print(correctTxt)
        
    print('accuracy=',accuracy,"/",len(testSamples),"=",accuracy/len(testSamples))


--------------------------------------------------
Iteration 50
Validation accuracy: 0.36079999774694443
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
expression: DCL            correct: 650  predicted: 100  
expression: MDXX           correct: 1520 predicted: 1000 
expression: MCCCXXIV       correct: 1324 predicted: 1323 
expression: DCCCL          correct: 850  predicted: 155  
expression: MLXIII         correct: 1063 predicted: 122  
expression: CCCXXXVI       correct: 336  predicted: 329  
expression: MCXCV          correct: 1195 predicted: 1555 
expression: MDCCCII        correct: 1802 predicted: 1922 
expression: CDLXXIX        correct: 479  predicted: 729  
expression: C              correct: 10

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\programs\miniconda3\envs\deep\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-ee0dce255be6>", line 18, in <module>
    preds = model.predict_classes(rowx, verbose=0)
  File "C:\programs\miniconda3\envs\deep\lib\site-packages\tensorflow\python\util\deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "C:\programs\miniconda3\envs\deep\lib\site-packages\tensorflow\python\keras\engine\sequential.py", line 453, in predict_classes
    proba = self.predict(x, batch_size=batch_size, verbose=verbose)
  File "C:\programs\miniconda3\envs\deep\lib\site-packages\tensorflow\python\keras\engine\training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\programs\miniconda3\envs\deep\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1569, in predict
    data_handler = data_

TypeError: object of type 'NoneType' has no len()